## CNN Text Classification with Keras

In [ ]:
%run Setup.ipynb

Load embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [123]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### Deeper 1D CNN
[Reference](https://github.com/richliao/textClassifier), [Source](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) and [Notes](http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)

Deeper Convolutional neural network: In [CNN for Sentence Classification](http://www.aclweb.org/anthology/D14-1181) (Yoon Kim, 2014), multiple filters have been applied. This can be implemented using Keras `Merge` Layer.

In [124]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

convs, filter_sizes = [], [1,3,5]
for fsz in filter_sizes:
    l_conv = Conv1D(filters=16,kernel_size=fsz,activation='relu',kernel_regularizer=regularizers.l2(0.01))(embedded_sequences)
    l_pool = MaxPooling1D(2)(l_conv)
    convs.append(l_pool)

l_merge = Concatenate(axis=1)(convs)
l_cov1= Conv1D(16, 3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_merge)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_drop1 = Dropout(0.1)(l_pool1)
l_cov2 = Conv1D(16, 3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_drop1)
l_pool2 = MaxPooling1D(17)(l_cov2) # global max pooling
l_flat = Flatten()(l_pool2)
l_dense = Dense(16, activation='relu')(l_flat)
preds = Dense(4, activation='softmax')(l_dense)

In [125]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [126]:
def step_cyclic(epoch):
    try:
        l_r, decay = 1.0, 0.0001
        if epoch%15==0:multiplier = 10
        else:multiplier = 1
        rate = float(multiplier * l_r * 1/(1 + decay * epoch))
        print("learning_rate",round(rate,4))
        return rate
    except Exception as e:
        print("Error in lr_schedule:",str(e))
        return float(1.0)
    
def initial_boost(epoch):
    if epoch==0: return float(6.0)
    else: return float(1.0)
        
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=256, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoints", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(step_cyclic)
#lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [127]:
model.summary()
print("Training Progress:")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=32,
          callbacks=[tensorboard, model_checkpoints, lr_schedule])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_17 (Embedding)        (None, 30, 200)      6108600     input_26[0][0]                   
__________________________________________________________________________________________________
conv1d_125 (Conv1D)             (None, 30, 16)       3216        embedding_17[0][0]               
__________________________________________________________________________________________________
conv1d_126 (Conv1D)             (None, 28, 16)       9616        embedding_17[0][0]               
__________________________________________________________________________________________________
conv1d_127

KeyboardInterrupt: 